## 순환신경망 구현 및 학습

In [2]:
import tensorflow as tf

## 하이퍼 파라미터

In [3]:
EPOCHS =10
NUM_WORDS = 10000

## 모델 정의

In [16]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.rnn = tf.keras.layers.SimpleRNN(32) #LSTM으로 바꿀수 있다.
        self.dense = tf.keras.layers.Dense(2, activation = 'softmax')
        
    def call(self, x, training = None, mask = None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

## 학습, 테스트 루프 정의

In [17]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [18]:
imdb = tf.keras.datasets.imdb
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words = NUM_WORDS)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, value = 0, padding='pre',maxlen=32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, value = 0, padding='pre',maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).shuffle(10000).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [19]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [20]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5600392818450928, Accuracy: 69.08399963378906, Test Loss: 0.4602614939212799, Test Accuracy: 78.06800079345703
Epoch 2, Loss: 0.365124374628067, Accuracy: 84.00800323486328, Test Loss: 0.4827243983745575, Test Accuracy: 77.7760009765625
Epoch 3, Loss: 0.22145937383174896, Accuracy: 91.29999542236328, Test Loss: 0.5859109163284302, Test Accuracy: 76.4280014038086
Epoch 4, Loss: 0.09457072615623474, Accuracy: 96.7719955444336, Test Loss: 0.8016008734703064, Test Accuracy: 75.1240005493164
Epoch 5, Loss: 0.03777633607387543, Accuracy: 98.90800476074219, Test Loss: 1.0113410949707031, Test Accuracy: 73.38400268554688
Epoch 6, Loss: 0.02365374006330967, Accuracy: 99.2959976196289, Test Loss: 1.234387993812561, Test Accuracy: 75.14399719238281
Epoch 7, Loss: 0.01827474683523178, Accuracy: 99.38399505615234, Test Loss: 1.3480955362319946, Test Accuracy: 75.06800079345703
Epoch 8, Loss: 0.014020723290741444, Accuracy: 99.58799743652344, Test Loss: 1.4265681505203247, Test Accu

# LSTM

## 모델 정의

In [22]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.rnn = tf.keras.layers.LSTM(32) #GRU, LSTM으로 바꿀수 있다.
        self.dense = tf.keras.layers.Dense(2, activation = 'softmax')
        
    def call(self, x, training = None, mask = None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

In [23]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.01274071168154478, Accuracy: 99.58799743652344, Test Loss: 1.6350195407867432, Test Accuracy: 71.25599670410156
Epoch 2, Loss: 0.01588459685444832, Accuracy: 99.48400115966797, Test Loss: 1.6220670938491821, Test Accuracy: 71.7959976196289
Epoch 3, Loss: 0.00945872813463211, Accuracy: 99.6760025024414, Test Loss: 1.6899470090866089, Test Accuracy: 72.56800079345703
Epoch 4, Loss: 0.009560158476233482, Accuracy: 99.67200469970703, Test Loss: 1.761764645576477, Test Accuracy: 72.57599639892578
Epoch 5, Loss: 0.009215191006660461, Accuracy: 99.70399475097656, Test Loss: 1.8536592721939087, Test Accuracy: 70.43600463867188
Epoch 6, Loss: 0.008503220975399017, Accuracy: 99.68399810791016, Test Loss: 1.799615502357483, Test Accuracy: 73.86399841308594
Epoch 7, Loss: 0.008684532716870308, Accuracy: 99.73999786376953, Test Loss: 1.8324933052062988, Test Accuracy: 71.80400085449219
Epoch 8, Loss: 0.009848978370428085, Accuracy: 99.62799835205078, Test Loss: 1.8405251502990723, 